In [ ]:
# Load the already combined dataset
import pandas as pd

# Load the final combined dataset
final_df = pd.read_parquet('workspace/data/hackernews_full_data.parquet')

print(f"Loaded dataset: {len(final_df):,} rows")
print(f"Columns: {list(final_df.columns)}")
print(f"Memory usage: {final_df.memory_usage(deep=True).sum() / 1024**2:.1f} MB")
print(f"Date range: {final_df['time'].min()} to {final_df['time'].max()}")
print(f"Score range: {final_df['score'].min()} to {final_df['score'].max()}")

final_df.head()


In [1]:
# CELL 1: Import libraries
import pandas as pd
import psycopg2
from tqdm import tqdm
import os
print("Libraries imported successfully!")

Libraries imported successfully!


In [2]:
# CELL 2: Connect to database
conn = psycopg2.connect(
    dbname=os.getenv('DB_NAME'),
    user=os.getenv('DB_USER'),
    password=os.getenv('DB_PASSWORD'),
    host=os.getenv('DB_HOST'),
    port=os.getenv('DB_PORT')
)
print("Connected to database successfully!")

Connected to database successfully!


In [3]:
# CELL 3: Get total count first
count_query = """
SELECT  COUNT(*) 
FROM "hacker_news"."items" 
WHERE score IS NOT NULL 
  AND title IS NOT NULL 
  AND by IS NOT NULL
  AND type = 'story'
"""

total_rows = pd.read_sql(count_query, conn).iloc[0, 0]
print(f"Total rows to process: {total_rows:,}")

/tmp/ipykernel_8998/276969155.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  total_rows = pd.read_sql(count_query, conn).iloc[0, 0]


Total rows to process: 4,902,536


In [4]:
# CELL 4: Set chunk parameters
chunk_size = 50000  # Adjust this if needed
num_chunks = (total_rows // chunk_size) + (1 if total_rows % chunk_size > 0 else 0)
print(f"Will process in {num_chunks} chunks of {chunk_size:,} rows each")


Will process in 99 chunks of 50,000 rows each


In [8]:
all_chunks = []
print("Starting chunked loading...")

# CELL 6: Load first chunk (test)
chunk_query = f"""
SELECT id, by, time, url, score, title, descendants
FROM "hacker_news"."items" 
WHERE score IS NOT NULL 
  AND title IS NOT NULL 
  AND by IS NOT NULL
  AND type = 'story'
ORDER BY time
LIMIT {chunk_size} OFFSET 0
"""

chunk_df = pd.read_sql(chunk_query, conn)
print(f"First chunk loaded: {len(chunk_df):,} rows")
print(f"Memory usage: {chunk_df.memory_usage(deep=True).sum() / 1024**2:.1f} MB")

# Save first chunk
chunk_filename = 'workspace/data/chunks/chunk_0.parquet'
chunk_df.to_parquet(chunk_filename, index=False)
all_chunks.append(chunk_filename)  # Store filename instead of dataframe
print(f"First chunk saved as {chunk_filename}")


Starting chunked loading...


/tmp/ipykernel_8998/2324593453.py:16: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  chunk_df = pd.read_sql(chunk_query, conn)


First chunk loaded: 50,000 rows
Memory usage: 15.0 MB
First chunk saved as workspace/data/chunks/chunk_0.parquet


In [11]:
# CELL 7: Load remaining chunks in loop
for i in range(1, num_chunks):
    offset = i * chunk_size
    
    chunk_query = f"""
    SELECT id, by, time, url, score, title, descendants
    FROM "hacker_news"."items" 
    WHERE score IS NOT NULL 
      AND title IS NOT NULL 
      AND by IS NOT NULL
      AND type = 'story'
    ORDER BY time
    LIMIT {chunk_size} OFFSET {offset}
    """
    
    try:
        chunk_df = pd.read_sql(chunk_query, conn)
        if len(chunk_df) > 0:
            # Save chunk immediately
            chunk_filename = f'workspace/data/chunks/chunk_{i}.parquet'
            chunk_df.to_parquet(chunk_filename, index=False)
            all_chunks.append(chunk_filename)
            print(f"Chunk {i+1}/{num_chunks} saved: {len(chunk_df):,} rows -> {chunk_filename}")
        else:
            print(f"Chunk {i+1} was empty, stopping")
            break
            
    except Exception as e:
        print(f"Error loading chunk {i+1}: {e}")
        break

print(f"Completed loading {len(all_chunks)} chunks")


/tmp/ipykernel_8998/3387744160.py:17: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  chunk_df = pd.read_sql(chunk_query, conn)


Chunk 2/99 saved: 50,000 rows -> workspace/data/chunks/chunk_1.parquet
Chunk 3/99 saved: 50,000 rows -> workspace/data/chunks/chunk_2.parquet
Chunk 4/99 saved: 50,000 rows -> workspace/data/chunks/chunk_3.parquet
Chunk 5/99 saved: 50,000 rows -> workspace/data/chunks/chunk_4.parquet
Chunk 6/99 saved: 50,000 rows -> workspace/data/chunks/chunk_5.parquet
Chunk 7/99 saved: 50,000 rows -> workspace/data/chunks/chunk_6.parquet
Chunk 8/99 saved: 50,000 rows -> workspace/data/chunks/chunk_7.parquet
Chunk 9/99 saved: 50,000 rows -> workspace/data/chunks/chunk_8.parquet
Chunk 10/99 saved: 50,000 rows -> workspace/data/chunks/chunk_9.parquet
Chunk 11/99 saved: 50,000 rows -> workspace/data/chunks/chunk_10.parquet
Chunk 12/99 saved: 50,000 rows -> workspace/data/chunks/chunk_11.parquet
Chunk 13/99 saved: 50,000 rows -> workspace/data/chunks/chunk_12.parquet
Chunk 14/99 saved: 50,000 rows -> workspace/data/chunks/chunk_13.parquet
Chunk 15/99 saved: 50,000 rows -> workspace/data/chunks/chunk_14.par

In [2]:
import pandas as pd
import glob
import os

print("Finding all chunk files...")
chunk_files = sorted(glob.glob('workspace/data/chunks/chunk_*.parquet'))
print(f"Found {len(chunk_files)} chunk files")

print("\nCombining all chunks...")
combined_chunks = []

for chunk_file in chunk_files:
    chunk_df = pd.read_parquet(chunk_file)
    combined_chunks.append(chunk_df)
    print(f"Loaded {chunk_file}: {len(chunk_df):,} rows")

final_df = pd.concat(combined_chunks, ignore_index=True)
print(f"\nFinal combined dataset: {len(final_df):,} rows")

# Save final dataset
output_file = 'workspace/data/hackernews_full_data.parquet'
final_df.to_parquet(output_file, index=False)
print(f"\nFinal dataset saved as {output_file}")

# Clean up individual chunk files
print("\nCleaning up chunk files...")
for chunk_file in chunk_files:
    try:
        os.remove(chunk_file)
        print(f"Removed {chunk_file}")
    except Exception as e:
        print(f"Could not remove {chunk_file}: {e}")

print("\nProcess completed! ✅")

Finding all chunk files...
Found 99 chunk files

Combining all chunks...
Loaded workspace/data/chunks/chunk_0.parquet: 50,000 rows
Loaded workspace/data/chunks/chunk_1.parquet: 50,000 rows
Loaded workspace/data/chunks/chunk_10.parquet: 50,000 rows
Loaded workspace/data/chunks/chunk_11.parquet: 50,000 rows
Loaded workspace/data/chunks/chunk_12.parquet: 50,000 rows
Loaded workspace/data/chunks/chunk_13.parquet: 50,000 rows
Loaded workspace/data/chunks/chunk_14.parquet: 50,000 rows
Loaded workspace/data/chunks/chunk_15.parquet: 50,000 rows
Loaded workspace/data/chunks/chunk_16.parquet: 50,000 rows
Loaded workspace/data/chunks/chunk_17.parquet: 50,000 rows
Loaded workspace/data/chunks/chunk_18.parquet: 50,000 rows
Loaded workspace/data/chunks/chunk_19.parquet: 50,000 rows
Loaded workspace/data/chunks/chunk_2.parquet: 50,000 rows
Loaded workspace/data/chunks/chunk_20.parquet: 50,000 rows
Loaded workspace/data/chunks/chunk_21.parquet: 50,000 rows
Loaded workspace/data/chunks/chunk_22.parquet

In [ ]:
# CELL 9: Save final dataset and cleanup
final_df.to_parquet('workspace/data/hackernews_full_data.parquet', index=False)
print("Final dataset saved as hackernews_full_data.parquet")

# Clean up individual chunk files
for chunk_file in all_chunks:
    try:
        os.remove(chunk_file)
        print(f"Removed {chunk_file}")
    except:
        print(f"Could not remove {chunk_file}")

print("Cleanup completed!")


In [3]:
# CELL 10: Verify final dataset
print("Final dataset info:")
print(f"Shape: {final_df.shape}")
print(f"Memory usage: {final_df.memory_usage(deep=True).sum() / 1024**2:.1f} MB")
print(f"Date range: {final_df['time'].min()} to {final_df['time'].max()}")
print(f"Score range: {final_df['score'].min()} to {final_df['score'].max()}")

Final dataset info:
Shape: (4902536, 7)
Memory usage: 1605.0 MB
Date range: 2006-10-09 19:21:51 to 2024-10-13 23:53:00
Score range: -1 to 6015


In [ ]:
# CELL 11: Close database connection
conn.close()
print("Database connection closed")
print("Data loading complete! ✅")

In [5]:
# CELL 12: Quick data preview
print("Sample data:")
final_df.head()



Sample data:


,id,by,time,url,score,title,descendants
0,1,pg,2006-10-09 19:21:51,http://ycombinator.com,57,Y Combinator,15.0
1,2,phyllis,2006-10-09 19:30:28,http://www.paulgraham.com/mit.html,16,A Student's Guide to Startups,0.0
2,3,phyllis,2006-10-09 19:40:33,http://www.foundersatwork.com/stevewozniak.html,7,Woz Interview: the early days of Apple,0.0
3,4,onebeerdave,2006-10-09 19:47:42,http://avc.blogs.com/a_vc/2006/10/the_nyc_deve...,5,NYC Developer Dilemma,0.0
4,5,perler,2006-10-09 19:51:04,http://www.techcrunch.com/2006/10/09/google-yo...,7,"Google, YouTube acquisition announcement could...",0.0


In [ ]:
# Convert SQL query to pandas operations
# Original SQL filters and selects specific columns with calculated fields

# Filter the data (note: type='story' and by IS NOT NULL already applied during data loading)
filtered_df = final_df[
    (final_df['score'] >= 0)  # score >= 0 filter
].copy()

# Add calculated columns
filtered_df['title_length'] = filtered_df['title'].str.len()
filtered_df['title_word_count'] = filtered_df['title'].str.strip().str.split().str.len()

# Select desired columns and sort by time descending
result_df = filtered_df[['id', 'score', 'title', 'title_length', 'title_word_count']].sort_values('time', ascending=False)

print(f"Filtered dataset: {len(result_df):,} rows")
print("\nSample results:")
result_df.head(10)
